CS524: Introduction to Optimization Lecture 23
======================================

### Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

#### October 25, 2024
--------------

## QAP – The second most famous problem
“Quadratic” Assignment Problem
- Set of facilities F 
- Set of locations L 
- $d_{ij}$ distance from i∈L to j∈L
- $f_{kl}$ flow from k∈F to l∈F

## QAP
$$ x_{ij} =   \left\{
\begin{array}{ll}
      1 & \text{Assign facility $i$ to location $j$} \\
      0 & \text{Otherwise} \\
\end{array} 
\right. \\ \\ $$

$$
min \Sigma_{k \in F} \Sigma_{i \in L} \Sigma_{I \in F} \Sigma_{j \in L} d_{ij} f_{kl} x_{ki} x_{Ij} \\ \\
\Sigma_{i \in F} x_{ij} = 1 \forall j \in L \\ \\
\Sigma_{j \in L} x_{ij} = 1 \forall i \in F \\ \\
x_{ij} \in \{0,1\} \forall i \in F, \forall j \in L
$$

## QAP
- $x_{ki} x_{lj}$ is nonlinear!
- What you really want it is to count $d_{ij} f_{kl}$ towards your objective if and only if you assign facility k→i and facility l→j
- There is one more “trick” you should know – when you are multiplying two binary variables...

## A Final Trick
- Modeling Trick:  Linearizing product of two binaries
$$z_{kilj} = 1 ⇔ x_{ki} = 1, x_{lj} = 1 ⇔ x_{ki} ∧ x_{lj} \\ 
z_{kilj} ≤ x_{ki} ∀ k ∈ F, i ∈ L, l ∈ F, j ∈ L \\
z_{kilj} ≤ x_{lj} ∀ k ∈ F, i ∈ L, l ∈ F, j ∈ L \\
z_{kilj} ≥ x_{ki} + x_{lj}−1 ∀ k ∈ F, i ∈ L, l ∈ F, j ∈ L$$

- This is a special case of $P_n ↔ (P_1 ∧ ··· ∧ P_k)$
$$\delta_n + k \geq 1 + \Sigma_{i=1}^k \delta_i, \delta_j \geq \delta_n, j = 1, ..., k $$

- We can also do it a different way using the slide of Trix
$$x_{ki} + x_{lj} -2 z_{kilj} \geq 0$$

**The final trick for linearizing the product of two binary variables is shown below.**

In [1]:
import sys
import numpy as np

from gamspy import Container, Problem, Sense, Sum, Options
import gamspy.math as gpm

In [2]:
options = Options(relative_optimality_gap=1e-7,absolute_optimality_gap=0,seed=612)
m = Container(options=options)

# DATA
L = m.addSet('L',records=[f'i{i+1}' for i in range(0,8)])
L1 = m.addAlias('L1',L)
L2 = m.addAlias('L2',L)
F = m.addSet('F',records=[f'k{i+1}' for i in range(0,8)])
F1 = m.addAlias('F1',F)
F2 = m.addAlias('F2',F)

dist=m.addParameter('dist',domain=[L1,L2],description="Distance")
flow=m.addParameter('flow',domain=[F1,F2],description="Flow")
xloc=m.addParameter('xloc',domain=[L],description="x location")
yloc=m.addParameter('yloc',domain=[L],description="y location")
xloc[L] = gpm.uniform(-10,10)
yloc[L] = gpm.uniform(-10,10) 
dist[L1,L2] = gpm.sqrt(gpm.sqr(xloc[L1]-xloc[L2]) + gpm.sqr(yloc[L1]-yloc[L2]))
flow[F1,F2] = gpm.Round(gpm.uniform(-50,50))

# MODEL

x = m.addVariable('x','binary',domain=[F,L])
 
assign_fac = m.addEquation('assign_fac', domain=F) 
assign_fac[F] = Sum(L, x[F,L]) == 1

assign_loc = m.addEquation('assign_loc', domain=L,)
assign_loc[:] = Sum(F, x[F,L]) == 1

qap_nonlinear = m.addModel('qap_nonlinear',
    equations=m.getEquations(),
    problem=Problem.MINLP,
    sense=Sense.MIN,
    objective=Sum([F1,L1,F2,L2], dist[L1,L2]*flow[F1,F2]*x[F1,L1]*x[F2,L2]),
)

qap_nonlinear.solve(options=Options(time_limit=30),solver="baron",solver_options={'ExtNLPsolver':'conopt'},output=sys.stdout)
# qap_nonlinear.solve(options=Options(time_limit=30),output=None)
print(f"nonlinear gap = {qap_nonlinear.objective_value - qap_nonlinear.objective_estimation}")
print(f"time = {qap_nonlinear.total_solver_time}\n")

--- Job _ae7f6df5-c618-44d5-911e-d86b44aba4d9.gms Start 10/25/24 09:14:54 48.1.0 612569ae DAX-DAC arm 64bit/macOS
--- Applying:
    /Users/ferris/.venvs/gamspy++/lib/python3.12/site-packages/gamspy_base/gmsprmun.txt
    /Users/ferris/Library/Preferences/GAMS/gamsconfig.yaml
--- GAMS Parameters defined
    MINLP baron
    Input /var/folders/yz/_rxq7zs56y5cnkx3mcnkr7txvxb3pw/T/tmp0_81h8bv/_ae7f6df5-c618-44d5-911e-d86b44aba4d9.gms
    Output /var/folders/yz/_rxq7zs56y5cnkx3mcnkr7txvxb3pw/T/tmp0_81h8bv/_ae7f6df5-c618-44d5-911e-d86b44aba4d9.lst
    ScrDir /var/folders/yz/_rxq7zs56y5cnkx3mcnkr7txvxb3pw/T/tmp0_81h8bv/tmpsksdr30v/
    SysDir /Users/ferris/.venvs/gamspy++/lib/python3.12/site-packages/gamspy_base/
    LogOption 3
    Trace /var/folders/yz/_rxq7zs56y5cnkx3mcnkr7txvxb3pw/T/tmp0_81h8bv/_ae7f6df5-c618-44d5-911e-d86b44aba4d9.txt
    License "/Users/ferris/Library/Application Support/GAMSPy/gamspy_license.txt"
    OptFile 1
    OptDir /var/folders/yz/_rxq7zs56y5cnkx3mcnkr7txvxb3pw/T/t

[MODEL - WARNING] The solve was interrupted! Solve status: ResourceInterrupt. For further information, see https://gamspy.readthedocs.io/en/latest/reference/gamspy._model.html#gamspy.SolveStatus.


nonlinear gap = 5766.122087674434
time = 30.188000109046698



In [3]:
# Now do a linear model
z = m.addVariable('z','binary',domain=[F1,L1,F2,L2])

# If all dist and flow are positive, you don't need trix_1 and trix_2
trix_1 = m.addEquation('trix_1', domain=[F1,L1,F2,L2])
trix_1[F1,L1,F2,L2]= z[F1,L1,F2,L2] <= x[F1,L1]

trix_2 = m.addEquation('trix_2', domain=[F1,L1,F2,L2])
trix_2[F1,L1,F2,L2]= z[F1,L1,F2,L2] <= x[F2,L2]

trix_3 = m.addEquation('trix_3', domain=[F1,L1,F2,L2])
trix_3[F1,L1,F2,L2]= z[F1,L1,F2,L2] >= x[F1,L1] + x[F2,L2] - 1

qap_linear = m.addModel('qap_linear',
    equations=[assign_fac, assign_loc, trix_1, trix_2, trix_3],
    problem=Problem.MIP,
    sense=Sense.MIN,
    objective=Sum([F1,L1,F2,L2], dist[L1,L2]*flow[F1,F2]*z[F1,L1,F2,L2] ),
)

# CPLEX is ok, gurobi is best, copt ok on first model
qap_linear.solve(options=Options(time_limit=30),solver="cplex",output=None)
print(f"linear gap = {qap_linear.objective_value - qap_linear.objective_estimation}")
print(f"time = {qap_linear.total_solver_time}\n")

# Previous model could have z positive with up=1, but following needs z binary
trix_4 = m.addEquation('trix_4', domain=[F1,L1,F2,L2])
trix_4[F1,L1,F2,L2]= x[F1,L1] + x[F2,L2] >= 2*z[F1,L1,F2,L2]


qap_linear = m.addModel('qap_linear',
    equations=[assign_fac, assign_loc, trix_3, trix_4],
    problem=Problem.MIP,
    sense=Sense.MIN,
    objective=Sum([F1,L1,F2,L2], dist[L1,L2]*flow[F1,F2]*z[F1,L1,F2,L2] ),
)

qap_linear.solve(options=Options(time_limit=30),solver="cplex",output=None)
print(f"linear (aggregate) gap = {qap_linear.objective_value - qap_linear.objective_estimation}")
print(f"time = {qap_linear.total_solver_time}\n")

linear gap = 0.0
time = 6.686999788507819

linear (aggregate) gap = 0.0
time = 6.529000494629145

